<a href="https://colab.research.google.com/github/graphy-young/endslab/blob/main/2022/research/openapi_nifos_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialize notebook

### Import requirements

In [ ]:
!pip install ipython-autotime pyblaze
%load_ext autotime

In [17]:
import requests, json
import pandas as pd
from pprint import pprint
import pyblaze.multiprocessing as xmp

time: 2.09 ms (started: 2022-06-30 04:50:32 +00:00)


## Define constants & variables

### Load connection info from Google Drive

In [ ]:
from google.colab import drive

drive_path = "/content/drive" 
drive.mount(drive_path)

json_path = f"{drive_path}/MyDrive/Colab Notebooks/ENDS_LAB/openapi.json"
open_api = json.load(open(json_path, 'r'))

### 산림청 국립산림과학원_청정넷_측정데이터 (AicanDustData)

In [ ]:
### Documentation -> https://www.data.go.kr/data/15080316/fileData.do

nifos_measurement = dict()
nifos_measurement["OPENAPI_URL"] = open_api["nifos_msmt"]["endpoint"]
nifos_measurement["API_KEY_ENCODED"] = open_api["nifos_msmt"]["key"] # Insert your Open API encoded key
nifos_measurement["API_KEY_DECODED"] = requests.utils.unquote(nifos_measurement["API_KEY_ENCODED"])
nifos_measurement["QUOTA_LIMIT"] = 180000

nifos_measurement["column_list"] = ["measured_datetime", # obsrt_dtm, 관측일시
                                "temperature", # obsrt_dtm, 관측_온도
                                "avoc_pm01", # obsrt_dtm, 관측_습도
                                "wind_speed", # obsrt_ws, 관측_풍속
                                "pm10", # obsrt_pm10_val, 관측_미세먼지
                                "pm25", # obsrt_pm25_val, 관측_초미세먼지
                                "avoc_pm10", # avoc_obsrt_pm10_val, AVOC_관측_미세먼지
                                "avoc_pm25", # avoc_obsrt_pm25_val, AVOC_관측_초미세먼지
                                "relative_humidity", # obsrt_hmdt, 관측_습도
                                "pm01", # obsrt_pm01_val, 관측_극초미세먼지
                                "station_code", # obsrr_tpcd, 관측소_구분코드
                                "wind_direction" # obsrt_wndrc_val, 관측_풍향
                                ]

nifos_measurement["parameters"] = {
    'serviceKey' : nifos_measurement["API_KEY_DECODED"], # API Authorizaiton key
    'pageNo' : '1', # Page number (Default: 1)
    'numOfRows' : str(nifos_measurement['QUOTA_LIMIT']), # Result row numbers per a page (Default: 10) ++ Rate exceed threshold 최대 약 18만건?
    'contentType' : 'JSON' # Response data type (xml, json / Default: xml)
    } # Make request as 'between' using "endDt" same format as startDt 
    #'startDt' : '202104010000', # yyyyMMddHHmmss (Default: None)

pprint(nifos_measurement)

### 산림청 국립산림과학원_청정넷_운영현황 (AicanDustData)

## Get observations data from Open API

In [ ]:
max_iter_num = 0

response = requests.get(AicanDustData["OPENAPI_URL"], params=AicanDustData["parameters"])
pprint(response)

response_json = json.loads(response.content)

result_code = response_json['resultCode']
result_message = response_json['resultMsg']
num_of_rows = int(response_json['numOfRows'])
page_no = int(response_json['pageNo'])
total_count = int(response_json['totalCount'])

data = pd.DataFrame.from_dict(response_json['items'])

print(f"Load {AicanDustData['QUOTA_LIMIT']} of {total_count} rows successfully with code {result_code}: {result_message}")

if AicanDustData['QUOTA_LIMIT'] < total_count:
  max_iter_num = (total_count // AicanDustData['QUOTA_LIMIT']) - 1
  if total_count % AicanDustData['QUOTA_LIMIT'] > 0:
    max_iter_num += 1
  for page_num in range(max_iter_num):
    AicanDustData["parameters"]["pageNo"] = str(int(AicanDustData["parameters"]["pageNo"]) + 1)
    response = requests.get(AicanDustData["OPENAPI_URL"], params=AicanDustData["parameters"])
    pprint(response)
    print(f"Load {response_json['numOfRows']} of {total_count} rows successfully with code {result_code}: {result_message}")
    response_json = json.loads(response.content)
    data = data.append(pd.DataFrame.from_dict(response_json['items']), ignore_index=True)
  else:
    AicanDustData["parameters"]["pageNo"] = "1"

# MEMO: prettify codes using divmod()

display(data)

In [ ]:
data.count()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data.to_csv('data.csv')
!cp data.csv "/content/drive/MyDrive/"

In [ ]:
!cp data.csv "/content/drive/MyDrive/"